In [1]:

import torch
import torch.nn.functional as F

In [2]:
a = torch.randn(1, 129, 88)
input_size = a.shape[2]
input_size = 88
hidden_size = 400

rnn = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, bidirectional=True, batch_first=True)
out, hidden = rnn(a) # out: [1, 129, 800], hidden: [2, 1, 400]
print(f'out.shape: {out.shape}')
print(f'hidden.shape: {hidden.shape}')
h_left = out[:, :, :hidden_size]
h_right = out[:, :, hidden_size:]

print(f'h_left.shape: {h_left.shape}')
print(f'h_right.shape: {h_right.shape}')


out.shape: torch.Size([1, 129, 800])
hidden.shape: torch.Size([2, 1, 400])
h_left.shape: torch.Size([1, 129, 400])
h_right.shape: torch.Size([1, 129, 400])


In [3]:
z_0 = torch.randn(1, 1, 100)

In [4]:
combiner = torch.nn.Linear(100, 400)
h_combined = combiner(z_0)
h_combined = .5 * (F.tanh(h_combined) + h_right)
print(f'h_combined.shape: {h_combined.shape}')

h_combined.shape: torch.Size([1, 129, 400])


In [5]:
def softplus(x):
    return torch.log(1 + torch.exp(x))

In [6]:
mu_linear = torch.nn.Linear(400, 100)
sigma_linear = torch.nn.Linear(400, 100)

mu = mu_linear(h_combined)
sigma = softplus(sigma_linear(h_combined))

print(f'mu.shape: {mu.shape}')
print(f'sigma.shape: {sigma.shape}')    


mu.shape: torch.Size([1, 129, 100])
sigma.shape: torch.Size([1, 129, 100])


In [7]:
z_1 = mu + sigma * torch.randn_like(mu)


In [8]:
#emission function
emission = torch.nn.Sequential(
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 88),
    torch.nn.Sigmoid()
)

#transition function
G = torch.nn.Sequential(
    torch.nn.Linear(100, 200), #gating unit
    torch.nn.ReLU(),
    torch.nn.Linear(200, 100), #proposed mean
    torch.nn.Sigmoid()
)

H = torch.nn.Sequential(
    torch.nn.Linear(100, 200), #proposed mean
    torch.nn.ReLU(),
    torch.nn.Linear(200, 100), #proposed mean
    torch.nn.Identity(),
)

mu_gated_linear = torch.nn.Linear(100, 100) #w_{mu_p} * z_{t-1} + b_{mu_p}
sigma_gated_linear = torch.nn.Sequential( #w_{sigma_p} * relu(h_t) + b_{sigma_p}
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.Softplus(),
)
    




In [9]:
emission(z_1).shape

torch.Size([1, 129, 88])

In [10]:
#forward pass of transition function
z_0 = torch.zeros(1, 1, 100)
g_out = G(z_0)
one_minus_g = 1 - g_out # 1 - g
mu_linear_out = mu_gated_linear(z_0) #w_{mu_p} * z_{t-1} + b_{mu_p}
elementwise_mu_out = mu_linear_out * one_minus_g
proposed_mean_out = H(z_0)
mu_generator = (proposed_mean_out * g_out) + elementwise_mu_out

sigma_generator = sigma_gated_linear(proposed_mean_out)


print(f'out.shape: {one_minus_g.shape}')
print(f'sigma_generator.shape: {sigma_generator.shape}')

out.shape: torch.Size([1, 1, 100])
sigma_generator.shape: torch.Size([1, 1, 100])


In [11]:
z_1 = mu_generator + sigma_generator * torch.randn_like(mu_generator)

In [12]:
from model import DVAE

import torch
import torch.nn.functional as F

In [13]:
model = DVAE()

In [14]:
x = torch.randn(3, 129, 88)

In [15]:
x_hat, mus_p_z, sigmas_p_z, mus_generator, sigmas_generators = model(x)

In [26]:
def kl_normal(qm, qv, pm, pv):
    """
    Computes the elem-wise KL divergence between two normal distributions KL(q || p) and
    sum over the last dimension

    Args:
        qm: tensor: (batch, dim): q mean
        qv: tensor: (batch, dim): q variance
        pm: tensor: (batch, dim): p mean
        pv: tensor: (batch, dim): p variance

    Return:
        kl: tensor: (batch,): kl between each sample
    """
    
    element_wise = 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    print(f'element_wise.shape: {element_wise.shape}')
    kl = element_wise.sum(-1)
    return kl

In [27]:
print(f'x_hat.shape: {x_hat.shape}')
print(f'mus_p_z.shape: {mus_p_z.shape}')
print(f'sigmas_p_z.shape: {sigmas_p_z.shape}')
print(f'mus_generator.shape: {mus_generator.shape}')
print(f'sigmas_generators.shape: {sigmas_generators.shape}')


x_hat.shape: torch.Size([3, 129, 88])
mus_p_z.shape: torch.Size([3, 129, 100])
sigmas_p_z.shape: torch.Size([3, 129, 100])
mus_generator.shape: torch.Size([3, 129, 100])
sigmas_generators.shape: torch.Size([3, 129, 100])


In [28]:
mu_z_1 = mus_p_z[:, 0, :]
sigma_z_1 = sigmas_p_z[:, 0, :]
print(f'mu_z_1.shape: {mu_z_1.shape}')
print(f'sigma_z_1.shape: {sigma_z_1.shape}')

mus_generator_1 = mus_generator[:, 0, :]
sigmas_generator_1 = sigmas_generators[:, 0, :]
print(f'mus_generator_1.shape: {mus_generator_1.shape}')
print(f'sigmas_generator_1.shape: {sigmas_generator_1.shape}')

single_kl = kl_normal(mu_z_1,
          sigma_z_1,
          mus_generator_1,
          sigmas_generator_1,
          )

print(f'single_kl: {single_kl}')

mu_z_1.shape: torch.Size([3, 100])
sigma_z_1.shape: torch.Size([3, 100])
mus_generator_1.shape: torch.Size([3, 100])
sigmas_generator_1.shape: torch.Size([3, 100])
element_wise.shape: torch.Size([3, 100])
single_kl: tensor([3.3891, 3.3859, 3.2899], grad_fn=<SumBackward1>)


In [29]:
bce = torch.nn.BCEWithLogitsLoss(reduction='none')

In [20]:
def log_bernoulli_with_logits(x, logits):
    """
    Computes the log probability of a Bernoulli given its logits

    Args:
        x: tensor: (batch, dim): Observation
        logits: tensor: (batch, dim): Bernoulli logits

    Return:
        log_prob: tensor: (batch,): log probability of each sample
    """
    log_prob = -bce(input=logits, target=x).sum(-1).sum(-1) #TODO: check if suming over t is correct
    return log_prob

In [21]:
recon_loss = -log_bernoulli_with_logits(x_hat, x)

In [22]:
#KL(q, p) for t=2 to T

kl_q_p_2 = kl_normal(mus_generator[:, 1:, :],
          sigmas_generators[:, 1:, :],
          mus_p_z[:, 1:, :],
          sigmas_p_z[:, 1:, :])

In [23]:
kl_q_p_2.sum(-1).shape

torch.Size([3])

In [24]:
single_kl

tensor([3.3891, 3.3859, 3.2899], grad_fn=<SumBackward1>)

In [25]:
nelbo_loss = recon_loss.mean() + kl_q_p_2.sum(-1).mean() + single_kl.mean()

In [1]:
from omegaconf import OmegaConf
import argparse
import os
import torch
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import wandb

from dataloader import MusicDataset
from model import DVAE 

/Users/fedra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
path = 'config.yaml'

config = OmegaConf.load(path)

In [3]:


dataset = MusicDataset(config.dataset)
dataloader = DataLoader(dataset, 
                        batch_size=config.train.batch_size, 
                        num_workers=config.train.num_workers,
                        pin_memory=True, #important for speed
                        shuffle=True)

device = torch.device('mps')
model = DVAE(input_dim=config.model.input_dim, 
                hidden_dim=config.model.hidden_dim,
                hidden_dim_em=config.model.hidden_dim_em, 
                hidden_dim_tr=config.model.hidden_dim_tr, 
                latent_dim=config.model.latent_dim)
model = model.to(device)

    
for i, (encodings, masks, sequence_lengths) in enumerate(dataloader):
    print(f'encodings.shape: {encodings.shape}'
          f'masks.shape: {masks.shape}'
          f'sequence_lengths.shape: {sequence_lengths.shape}')
    encodings = encodings.to(device)
    masks = masks.to(device)
    sequence_lengths = sequence_lengths.to(device)
    x_hat, mus_p_z, sigmas_p_z, mus_generator, sigmas_generators = model(encodings)
    

/Users/fedra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/fedra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/fedra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/fedra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compi

encodings.shape: torch.Size([64, 129, 88])masks.shape: torch.Size([64, 129, 88])sequence_lengths.shape: torch.Size([64])
encodings.shape: torch.Size([64, 129, 88])masks.shape: torch.Size([64, 129, 88])sequence_lengths.shape: torch.Size([64])
encodings.shape: torch.Size([64, 129, 88])masks.shape: torch.Size([64, 129, 88])sequence_lengths.shape: torch.Size([64])
encodings.shape: torch.Size([37, 129, 88])masks.shape: torch.Size([37, 129, 88])sequence_lengths.shape: torch.Size([37])


In [4]:
print(f'x_hat.shape: {x_hat.shape}')
print(f'mus_p_z.shape: {mus_p_z.shape}')
print(f'sigmas_p_z.shape: {sigmas_p_z.shape}')
print(f'mus_generator.shape: {mus_generator.shape}')
print(f'sigmas_generators.shape: {sigmas_generators.shape}')


x_hat.shape: torch.Size([37, 129, 88])
mus_p_z.shape: torch.Size([37, 129, 100])
sigmas_p_z.shape: torch.Size([37, 129, 100])
mus_generator.shape: torch.Size([37, 129, 100])
sigmas_generators.shape: torch.Size([37, 129, 100])
